## Chapter 1.2: Data Structure and Manipulation
***
In the previous tutorial we saw how we can create a model by calling a few lines of code from imported data. However modelling tends to be an iterative process. Here we will explore the tools that `GemPy` with the help of `pandas` offers to modify the input data of a model.

There are 5 main  funtion "types" in GemPy:

- *create*: 
    - create new objects
    - return the objects
 
- *set*
    - set given values **inplace**

- *update*
    - update dataframe or other attribute from other object or many objects. Usually this object is not passed as argument (this is the main difference with map)

- *map*
    - update dataframe (so far mainly df) or other attribute from an object to another object.
    - Completelly directed. One attribute/property is updated by another one.
    - In general, we map_from so the method is in the mutated object.

- *get*
    - return an image of the object
    
The intention is that a function/method that does not fall in any of these categories has a name (verb in principle) self explanatory. 


As always we start importing the usual packages and reading expample data:

In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../../..")

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib notebook

#%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import pandas as pn
import matplotlib.pyplot as plt

## Series

Series is the object that contains the properties associated with each independent scalar field. Right now it is simply the order of the series (which is infered by the index order). But in the future will be add the unconformity relation or perhaps the type of interpolator

Series and Faults classes are quite entagled since fauls are a type of series

In [2]:
faults = gp.Faults()
series = gp.Series(faults)
series.df

,order_series,BottomRelation,isActive
Default series,1,Erosion,False


We can modify the series bt using `set_series_index`:

In [3]:
series.set_series_index(['foo', 'foo2', 'foo5', 'foo7'])
series

,order_series,BottomRelation,isActive
foo,1,Erosion,False
foo2,2,Erosion,NaN
foo5,3,Erosion,NaN
foo7,4,Erosion,NaN


The index of series are pandas categories. These provides quite handy backend functionality (see pandas.Categorical).

In [4]:
series.df.index

CategoricalIndex(['foo', 'foo2', 'foo5', 'foo7'], categories=['foo', 'foo2', 'foo5', 'foo7'], ordered=False, dtype='category')

For adding new series:

In [5]:
series.add_series('foo3')
series

,order_series,BottomRelation,isActive
foo,1,Erosion,False
foo2,2,Erosion,NaN
foo5,3,Erosion,NaN
foo7,4,Erosion,NaN
foo3,5,Erosion,NaN


Delete series

In [6]:
series.delete_series('foo3')
series

,order_series,BottomRelation,isActive
foo,1,Erosion,False
foo2,2,Erosion,NaN
foo5,3,Erosion,NaN
foo7,4,Erosion,NaN


Rename series:

In [7]:
series.rename_series({'foo':'boo'})
series

,order_series,BottomRelation,isActive
boo,1,Erosion,False
foo2,2,Erosion,NaN
foo5,3,Erosion,NaN
foo7,4,Erosion,NaN


Reorder series:

In [8]:
series.reorder_series(['foo2', 'boo', 'foo7', 'foo5'])
series

,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN


### Faults

The *df faults* is used to charectirize which *mathematical series* behave as fault and if mentioned faults are finite or infinite. Both df should get updated automatically as we modify the series object linked to the fault object (by passing it wehn a Series object is created).

In [9]:
faults

,isFault,isFinite
foo2,False,False
boo,False,False
foo7,False,False
foo5,False,False


Finally we have the *faults relations df* which captures which *mathematical series* a given fault offset in order to reproduce complex faulting networks

In [10]:
faults.faults_relations_df

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,False,False
foo7,False,False,False,False
foo5,False,False,False,False


We can use `set_is_fault` to choose which of our series are faults:

In [11]:
faults.set_is_fault(['boo'])

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False


Similar thing for the fault relations:

In [13]:
fr = np.zeros((4, 4))
fr[2, 2] = True
fr[1, 2] = True

faults.set_fault_relation(fr)

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,True,False
foo7,False,False,False,False
foo5,False,False,False,False


Now if we change the series df and we update the series already defined will conserve their values while the new ones will be set to false:

In [13]:
series.add_series('foo20')


,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN
foo20,5,Erosion,NaN


In [14]:
series

,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN
foo20,5,Erosion,NaN


In [15]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [16]:
faults.faults_relations_df

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


When we add new series the values switch  to NaN. We will be careful not having any nan in the DataFrames or we will raise errors down the line.

In [17]:
faults.set_is_fault()

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [18]:
faults.set_fault_relation()

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


### Surfaces:

The *df* surfaces contains three properties. *id* refers to the order of the surfaces on the sequential pile, i.e. the strict order of computation. *values* on the other hand is the final value that each voxel will have after discretization. This may be useful for example in the case we want to map a specific geophysical property (such as density) to a given unity. By default both are the same since to discretize lithological units the value is arbitrary.

#### From an empty df

The Surfaces class needs to have an associate series object. This will limit the name of the series since they are a pandas.Category

In [19]:
surfaces = gp.Surfaces(series)

We can set any number of formations by passing a list with the names. By default they will take the name or the first series.

In [20]:
surfaces.set_surfaces_names(['foo', 'foo2', 'foo5'])


,surface,series,order_surfaces,isActive,color,id
0,foo,foo20,1,nan,#015482,1
1,foo2,foo20,2,nan,#9f0052,2
2,foo5,foo20,3,nan,#ffbe00,3


In [21]:
series

,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN
foo20,5,Erosion,NaN


We can add new formations:

In [22]:
surfaces.add_surface(['feeeee'])
surfaces

,surface,series,order_surfaces,isActive,color,id
0,foo,foo20,1,nan,#015482,1
1,foo2,foo20,2,nan,#9f0052,2
2,foo5,foo20,3,nan,#ffbe00,3
3,feeeee,foo20,4,nan,#728f02,4


The column formation is also a pandas.Categories. This will be important for the Data clases (surface_points and Orientations)

In [23]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

In [24]:
surfaces

,surface,series,order_surfaces,isActive,color,id
0,foo,foo20,1,nan,#015482,1
1,foo2,foo20,2,nan,#9f0052,2
2,foo5,foo20,3,nan,#ffbe00,3
3,feeeee,foo20,4,nan,#728f02,4


### Set values

To set the values we do it with the following method

In [25]:
surfaces.set_surfaces_values([2,2,2,5])

,surface,series,order_surfaces,color,id,value_0
0,foo,foo20,1,#015482,1,2
1,foo2,foo20,2,#9f0052,2,2
2,foo5,foo20,3,#ffbe00,3,2
3,feeeee,foo20,4,#728f02,4,5


In [26]:
surfaces

,surface,series,order_surfaces,color,id,value_0
0,foo,foo20,1,#015482,1,2
1,foo2,foo20,2,#9f0052,2,2
2,foo5,foo20,3,#ffbe00,3,2
3,feeeee,foo20,4,#728f02,4,5


#### Set values with a given name:

We can give specific names to the properties (i.e. density)

In [27]:
surfaces.add_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])

,surface,series,order_surfaces,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,#015482,1,2,2,2
1,foo2,foo20,2,#9f0052,2,2,2,2
2,foo5,foo20,3,#ffbe00,3,2,2,1
3,feeeee,foo20,4,#728f02,4,5,6,8


In [28]:
surfaces

,surface,series,order_surfaces,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,#015482,1,2,2,2
1,foo2,foo20,2,#9f0052,2,2,2,2
2,foo5,foo20,3,#ffbe00,3,2,2,1
3,feeeee,foo20,4,#728f02,4,5,6,8


### Delete formations values

To delete a full propery:

In [29]:
surfaces.delete_surface_values(['val_foo', 'value_0'])

True

#### One of the formations must be set be the basement:

In [30]:
surfaces.set_basement()
surfaces

,surface,series,order_surfaces,color,id,val2_foo
0,foo,foo20,1,#015482,1,2
1,foo2,foo20,2,#9f0052,2,2
2,foo5,foo20,3,#ffbe00,3,1
3,feeeee,foo20,4,#728f02,4,8


#### Set formation values

We can also use set values instead adding. This will delete the previous properties and add the new one

In [31]:
surfaces.set_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])
surfaces

,surface,series,order_surfaces,color,id,val_foo,val2_foo
0,foo,foo20,1,#015482,1,2,2
1,foo2,foo20,2,#9f0052,2,2,2
2,foo5,foo20,3,#ffbe00,3,2,1
3,feeeee,foo20,4,#728f02,4,6,8


The last property is the correspondant series that each formation belong to. `series` and `formation` are pandas categories. To get a overview of what this mean check https://pandas.pydata.org/pandas-docs/stable/categorical.html.

In [32]:
surfaces.df['series']

0    foo20
1    foo20
2    foo20
3    foo20
Name: series, dtype: category
Categories (5, object): [foo2, boo, foo7, foo5, foo20]

In [33]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

### Map series to formation

To map a series to a formation we can do it by passing a dict:

In [34]:
surfaces

,surface,series,order_surfaces,color,id,val_foo,val2_foo
0,foo,foo20,1,#015482,1,2,2
1,foo2,foo20,2,#9f0052,2,2,2
2,foo5,foo20,3,#ffbe00,3,2,1
3,feeeee,foo20,4,#728f02,4,6,8


In [35]:
series

,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN
foo20,5,Erosion,NaN


If a series does not exist in the `Series` object, we rise a warning and we set those formations to nans

In [36]:
d =  {"foo7":'foo', "booX": ('foo2','foo5', 'fee')}

In [37]:
surfaces.map_series(d)

,surface,series,order_surfaces,color,id,val_foo,val2_foo
0,foo,foo7,1,#015482,1,2,2
1,foo2,foo20,1,#9f0052,2,2,2
2,foo5,foo20,2,#ffbe00,3,2,1
3,feeeee,foo20,3,#728f02,4,6,8


In [38]:
surfaces.map_series({"foo7":'foo', "boo": ('foo2','foo5', 'fee')})

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,foo2,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#ffbe00,2,2,1
0,foo,foo7,1,#015482,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


In [39]:
surfaces

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,foo2,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#ffbe00,2,2,1
0,foo,foo7,1,#015482,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


An advantage of categories is that they are order so no we can tidy the df by series and formation

### Modify surface name

In [40]:
surfaces.rename_surfaces({'foo2':'lala'})

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,lala,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#ffbe00,2,2,1
0,foo,foo7,1,#015482,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


In [41]:
surfaces

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,lala,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#ffbe00,2,2,1
0,foo,foo7,1,#015482,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


In [42]:
surfaces.df.loc[2, 'val_foo'] = 22

In [43]:
surfaces

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,lala,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#ffbe00,2,22,1
0,foo,foo7,1,#015482,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


### Modify surface color

The surfaces DataFrame also contains a column for the color in which the surfaces are displayed. To change the color, call 

In [44]:
surfaces.colors.change_colors()

Click to select new colors.


This allow to change the colors interactively. If you already know which colors you want to use, you can also update them with a dictionary mapping the surface name to a hex color string:

In [45]:
new_colors={'foo':'#ff8000', 'foo5':'#4741be'}
surfaces.colors.change_colors(new_colors)

,surface,series,order_surfaces,color,id,val_foo,val2_foo
1,lala,boo,1,#9f0052,1,2,2
2,foo5,boo,2,#4741be,2,22,1
0,foo,foo7,1,#ff8000,3,2,2
3,feeeee,foo20,1,#728f02,4,6,8


# Data
#### surface_points
These two DataFrames (df from now on) will contain the individual information of each point at an interface or orientation. Some properties of this table are mapped from the *df* below.

In [46]:
surface_points = gp.SurfacePoints(surfaces)
#orientations = gp.Orientations()

In [47]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth


In [48]:
surface_points.set_surface_points(pn.DataFrame(np.random.rand(6,3)), ['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,NaN,NaN,1,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,NaN,NaN,1,0.000001
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,NaN,NaN,1,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,NaN,NaN,1,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,NaN,NaN,1,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,NaN,NaN,1,0.000001


In [49]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,NaN,NaN,1,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,NaN,NaN,1,0.000001
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,NaN,NaN,1,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,NaN,NaN,1,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,NaN,NaN,1,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,NaN,NaN,1,0.000001


In [50]:
surface_points.map_data_from_surfaces(surfaces, 'series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,foo7,NaN,1,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,boo,NaN,1,0.000001
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,boo,NaN,1,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,boo,NaN,1,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,boo,NaN,1,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,foo20,NaN,1,0.000001


In [51]:
surface_points.map_data_from_surfaces(surfaces, 'id')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,foo7,3,1,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,boo,2,1,0.000001
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,boo,1,1,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,boo,2,1,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,boo,1,1,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,foo20,4,1,0.000001


In [52]:
series

,order_series,BottomRelation,isActive
foo2,1,Erosion,NaN
boo,2,Erosion,False
foo7,3,Erosion,NaN
foo5,4,Erosion,NaN
foo20,5,Erosion,NaN


In [53]:
surface_points.map_data_from_series(series, 'order_series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,foo7,3,3,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,boo,2,2,0.000001
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,boo,1,2,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,boo,2,2,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,boo,1,2,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,foo20,4,5,0.000001


In [54]:
surface_points.sort_table()
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
2,0.032254,0.758358,0.178765,NaN,NaN,NaN,lala,boo,1,2,0.000001
4,0.397927,0.892439,0.159299,NaN,NaN,NaN,lala,boo,1,2,0.000001
1,0.669564,0.123407,0.604744,NaN,NaN,NaN,foo5,boo,2,2,0.000001
3,0.855237,0.437662,0.076444,NaN,NaN,NaN,foo5,boo,2,2,0.000001
0,0.555274,0.597690,0.702083,NaN,NaN,NaN,foo,foo7,3,3,0.000001
5,0.433711,0.978794,0.777369,NaN,NaN,NaN,feeeee,foo20,4,5,0.000001


In [55]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


### Orientations

In [56]:
orientations = gp.Orientations(surfaces)

In [57]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth


### Set values passing pole vectors:

In [58]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            np.random.rand(6,3),
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [59]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,3.438991,6.085507,1.657785,NaN,NaN,NaN,0.964985,0.896342,0.305502,72.211627,47.112020,1,foo,NaN,NaN,1,0.01
1,6.848263,7.453725,0.552474,NaN,NaN,NaN,0.176505,0.995761,0.630534,50.910446,10.051623,1,foo5,NaN,NaN,1,0.01
2,6.293600,4.149626,4.106191,NaN,NaN,NaN,0.914480,0.440870,0.007921,89.546139,64.261320,1,lala,NaN,NaN,1,0.01
3,5.199200,9.595404,6.468163,NaN,NaN,NaN,0.242569,0.264632,0.331538,70.637873,42.509257,1,foo5,NaN,NaN,1,0.01
4,5.072477,1.342326,6.998112,NaN,NaN,NaN,0.027056,0.989985,0.939891,19.966762,1.565476,1,lala,NaN,NaN,1,0.01
5,7.999344,7.247462,7.720903,NaN,NaN,NaN,0.053153,0.650580,0.602716,52.935345,4.670768,1,feeeee,NaN,NaN,1,0.01


### Set values pasing orientation data: azimuth, dip, pole (dip direction)

In [60]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            orientation = np.random.rand(6,3)*20,
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [61]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,9.997177,1.075890,9.290347,NaN,NaN,NaN,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,NaN,NaN,1,0.01
1,7.458344,5.431569,8.650059,NaN,NaN,NaN,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,NaN,NaN,1,0.01
2,9.836584,8.902736,3.370796,NaN,NaN,NaN,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,NaN,NaN,1,0.01
3,8.472255,0.595812,5.207201,NaN,NaN,NaN,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,NaN,NaN,1,0.01
4,9.037647,3.018788,5.518122,NaN,NaN,NaN,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,NaN,NaN,1,0.01
5,6.751707,8.529431,4.131771,NaN,NaN,NaN,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,NaN,NaN,1,0.01


### Mapping data from the other df

In [62]:
orientations.map_data_from_surfaces(surfaces, 'series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,9.997177,1.075890,9.290347,NaN,NaN,NaN,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,foo7,NaN,1,0.01
1,7.458344,5.431569,8.650059,NaN,NaN,NaN,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,boo,NaN,1,0.01
2,9.836584,8.902736,3.370796,NaN,NaN,NaN,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,boo,NaN,1,0.01
3,8.472255,0.595812,5.207201,NaN,NaN,NaN,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,boo,NaN,1,0.01
4,9.037647,3.018788,5.518122,NaN,NaN,NaN,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,boo,NaN,1,0.01
5,6.751707,8.529431,4.131771,NaN,NaN,NaN,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,foo20,NaN,1,0.01


In [63]:
orientations.map_data_from_surfaces(surfaces, 'id')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,9.997177,1.075890,9.290347,NaN,NaN,NaN,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,foo7,3,1,0.01
1,7.458344,5.431569,8.650059,NaN,NaN,NaN,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,boo,2,1,0.01
2,9.836584,8.902736,3.370796,NaN,NaN,NaN,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,boo,1,1,0.01
3,8.472255,0.595812,5.207201,NaN,NaN,NaN,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,boo,2,1,0.01
4,9.037647,3.018788,5.518122,NaN,NaN,NaN,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,boo,1,1,0.01
5,6.751707,8.529431,4.131771,NaN,NaN,NaN,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,foo20,4,1,0.01


In [64]:
orientations.map_data_from_series(series, 'order_series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,9.997177,1.075890,9.290347,NaN,NaN,NaN,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,foo7,3,3,0.01
1,7.458344,5.431569,8.650059,NaN,NaN,NaN,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,boo,2,2,0.01
2,9.836584,8.902736,3.370796,NaN,NaN,NaN,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,boo,1,2,0.01
3,8.472255,0.595812,5.207201,NaN,NaN,NaN,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,boo,2,2,0.01
4,9.037647,3.018788,5.518122,NaN,NaN,NaN,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,boo,1,2,0.01
5,6.751707,8.529431,4.131771,NaN,NaN,NaN,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,foo20,4,5,0.01


In [65]:
orientations.update_annotations()

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth,annotations
0,9.997177,1.075890,9.290347,NaN,NaN,NaN,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,foo7,3,3,0.01,"${\bf{x}}_{\beta \,{\bf{3}},0}$"
1,7.458344,5.431569,8.650059,NaN,NaN,NaN,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,boo,2,2,0.01,"${\bf{x}}_{\beta \,{\bf{2}},0}$"
2,9.836584,8.902736,3.370796,NaN,NaN,NaN,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,boo,1,2,0.01,"${\bf{x}}_{\beta \,{\bf{1}},0}$"
3,8.472255,0.595812,5.207201,NaN,NaN,NaN,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,boo,2,2,0.01,"${\bf{x}}_{\beta \,{\bf{2}},1}$"
4,9.037647,3.018788,5.518122,NaN,NaN,NaN,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,boo,1,2,0.01,"${\bf{x}}_{\beta \,{\bf{1}},1}$"
5,6.751707,8.529431,4.131771,NaN,NaN,NaN,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,foo20,4,5,0.01,"${\bf{x}}_{\beta \,{\bf{4}},0}$"


### Grid

In [66]:
grid = gp.Grid()
grid.create_regular_grid([0,10,0,10,0,10], [50,50,50])

In [67]:
grid.values

array([[0.1, 0.1, 0.1],
       [0.1, 0.1, 0.3],
       [0.1, 0.1, 0.5],
       ...,
       [9.9, 9.9, 9.5],
       [9.9, 9.9, 9.7],
       [9.9, 9.9, 9.9]])

#### Rescaling Data

In [68]:
rescaling = gp.RescaledData(surface_points, orientations, grid)


In [69]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
2,0.032254,0.758358,0.178765,0.250100,0.311704,0.274076,lala,boo,1,2,0.000001
4,0.397927,0.892439,0.159299,0.268448,0.318431,0.273099,lala,boo,1,2,0.000001
1,0.669564,0.123407,0.604744,0.282078,0.279844,0.295450,foo5,boo,2,2,0.000001
3,0.855237,0.437662,0.076444,0.291394,0.295612,0.268942,foo5,boo,2,2,0.000001
0,0.555274,0.597690,0.702083,0.276343,0.303642,0.300334,foo,foo7,3,3,0.000001
5,0.433711,0.978794,0.777369,0.270243,0.322764,0.304111,feeeee,foo20,4,5,0.000001


In [70]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth,annotations
0,9.997177,1.075890,9.290347,0.750100,0.327636,0.731258,0.067235,0.283329,0.961402,16.850929,13.349563,1.004535,foo,foo7,3,3,0.01,"${\bf{x}}_{\beta \,{\bf{3}},0}$"
1,7.458344,5.431569,8.650059,0.622712,0.546187,0.699131,0.308253,4.040686,12.302105,18.232359,4.362490,12.952372,foo5,boo,2,2,0.01,"${\bf{x}}_{\beta \,{\bf{2}},0}$"
2,9.836584,8.902736,3.370796,0.742042,0.720356,0.434239,0.004696,0.061700,0.529112,6.670350,4.352427,0.532718,lala,boo,1,2,0.01,"${\bf{x}}_{\beta \,{\bf{1}},0}$"
3,8.472255,0.595812,5.207201,0.673586,0.303548,0.526382,0.042819,0.128794,5.335396,1.457209,18.389983,5.337122,foo5,boo,2,2,0.01,"${\bf{x}}_{\beta \,{\bf{2}},1}$"
4,9.037647,3.018788,5.518122,0.701955,0.425123,0.541983,0.034576,0.133016,12.723346,0.618878,14.570796,12.724088,lala,boo,1,2,0.01,"${\bf{x}}_{\beta \,{\bf{1}},1}$"
5,6.751707,8.529431,4.131771,0.587255,0.701625,0.472422,0.570343,1.719373,14.474512,7.133537,18.351503,14.587427,feeeee,foo20,4,5,0.01,"${\bf{x}}_{\beta \,{\bf{4}},0}$"


### Additional Data

In [71]:
ad = gp.AdditionalData(surface_points, orientations, grid, faults, surfaces, rescaling)


In [72]:
ad

values
Structure isLith                                                                    True
          isFault                                                                   True
          number faults                                                                1
          number surfaces                                                              4
          number series                                                                5
          number surfaces per series                                     [0, 2, 1, 0, 1]
          len surfaces surface_points                                       [2, 2, 1, 1]
          len series surface_points                                      [0, 4, 1, 0, 1]
          len series orientations                                        [0, 4, 1, 0, 1]
Options   dtype                                                                  float64
          output                                                                 geology
          theano_optimizer                                                  fast_compile
          device                                                                     cpu
          verbosity                                                                 None
Kriging   range                                                                  17.3205
          $C_o$                                                                  7.14286
          drift equations                                                [3, 3, 3, 3, 3]
          nugget grad                                                               0.01
          nugget scalar                                                            1e-06
Rescaling rescaling factor                                                       19.9298
          centers                      [5.014715457330326, 4.513071395395816, 4.68339...

In [73]:
ad.structure_data

,values
isLith,True
isFault,True
number faults,1
number surfaces,4
number series,5
number surfaces per series,"[0, 2, 1, 0, 1]"
len surfaces surface_points,"[2, 2, 1, 1]"
len series surface_points,"[0, 4, 1, 0, 1]"
len series orientations,"[0, 4, 1, 0, 1]"


In [74]:
ad.options

,values
dtype,float64
output,geology
theano_optimizer,fast_compile
device,cpu
verbosity,None


In [75]:
ad.options.df

,dtype,output,theano_optimizer,device,verbosity
values,float64,geology,fast_compile,cpu,None


In [76]:
ad.options.df.dtypes

dtype               category
output              category
theano_optimizer    category
device              category
verbosity             object
dtype: object

In [77]:
ad.kriging_data

,values
range,17.3205
$C_o$,7.14286
drift equations,"[3, 3, 3, 3, 3]"
nugget grad,0.01
nugget scalar,1e-06


In [78]:
ad.rescaling_data

,values
rescaling factor,19.9298
centers,"[5.014715457330326, 4.513071395395816, 4.68339..."


### Interpolator

In [79]:
interp = gp.InterpolatorModel(surface_points, orientations, grid, surfaces, series, faults, ad)

In [80]:
interp.compile_th_fn_geo()

Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  1
Compilation Done!


In [81]:
interp.print_theano_shared()

len sereies i [0 2]
len sereies o [0 4]
len sereies w [ 0 17]
n surfaces per series [0 2]
n universal eq [3]
is finite [0 0 0 0 0]
is erosion [0]
is onlap [0]
